In [ ]:
"""
# FELIN Launch Vehicle Optimization with Environmental Objectives
# Using CMA-ES with Augmented Lagrangian and SLSQP
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openmdao.api as om
import cma
from scipy.optimize import minimize
import time
import warnings
warnings.filterwarnings('ignore')

# Import FELIN modules
from Launch_vehicle_Group import LauncherVehicleGroup
import result_vizualization

# Set print options
np.set_printoptions(threshold=100, precision=3)
print("FELIN Environmental Optimization Framework Loaded")

In [ ]:
"""
## Problem Definition
Minimize ESA environmental score subject to:
- ΔV ≥ 7800 m/s (reach orbital velocity)
- Altitude ≥ 300 km (reach target altitude)
- GLOW ≤ 500 t (launcher size constraint)
"""

# Design variable bounds
lowerbnd = np.array([
    230., 40.,           # Propellant masses (t)
    4.5, 4.5,           # O/F ratios
    0.10, 0.40, 0.05,   # Stage 1 materials
    0.10, 0.40, 0.05,   # Stage 2 materials
    0.40, 0.20, 0.05,   # Engine materials
    0., 5., 0.2,        # Trajectory params 1
    5., 5., 1.,         # Trajectory params 2
    -30., -30.          # Command
])

upperbnd = np.array([
    380., 80.,          # Propellant masses (t)
    6.0, 6.0,           # O/F ratios
    0.40, 0.70, 0.20,   # Stage 1 materials
    0.40, 0.70, 0.20,   # Stage 2 materials
    0.70, 0.40, 0.20,   # Engine materials
    5., 15., 0.4,       # Trajectory params 1
    10., 15., 5.,       # Trajectory params 2
    70., 70.            # Command
])

# Performance requirements
MIN_DELTA_V = 7800.0  # m/s
MIN_ALTITUDE = 300000.0  # m
MAX_GLOW = 500000.0  # kg

print(f"Design variables: {len(lowerbnd)}")
print(f"ΔV requirement: {MIN_DELTA_V} m/s")
print(f"Altitude requirement: {MIN_ALTITUDE/1000} km")

In [ ]:
"""
## Baseline Design
Starting from FELIN's original configuration
"""

x_baseline = np.array([
    250., 50.6,         # Propellant masses
    5.5, 5.5,           # O/F ratios
    0.10, 0.70, 0.20,   # Stage 1 materials
    0.10, 0.70, 0.20,   # Stage 2 materials
    0.60, 0.30, 0.10,   # Engine materials
    2.72, 10., 0.293,   # Trajectory
    5., 10., 1.,        # More trajectory
    30., -20.           # Command
])

# Normalize
x_baseline_norm = (x_baseline - lowerbnd) / (upperbnd - lowerbnd)
print(f"Baseline normalized: {x_baseline_norm[:4]}")  # Show first 4 values

In [ ]:
"""
## Fixed Point Iteration for Dynamic Pressure
Converges Pdyn_max for consistent structural sizing
"""

def run_fpi(x_norm, lowerbnd, upperbnd, prob):
    """Fixed Point Iteration with environmental evaluation"""
    
    # Denormalize
    x = lowerbnd + (upperbnd - lowerbnd) * x_norm
    
    # Set design variables
    prob['Prop_mass_stage_1'] = x[0] * 1e3
    prob['Prop_mass_stage_2'] = x[1] * 1e3
    prob['OF_stage_1'] = x[2]
    prob['OF_stage_2'] = x[3]
    
    # Material fractions
    prob['raw_cfrp_s1'] = x[4]
    prob['raw_aluminum_s1'] = x[5]
    prob['raw_steel_s1'] = x[6]
    prob['raw_cfrp_s2'] = x[7]
    prob['raw_aluminum_s2'] = x[8]
    prob['raw_steel_s2'] = x[9]
    prob['raw_nickel_eng'] = x[10]
    prob['raw_steel_eng'] = x[11]
    prob['raw_titanium_eng'] = x[12]
    
    # Trajectory
    prob['thetacmd_i'] = x[13]
    prob['thetacmd_f'] = x[14]
    prob['ksi'] = x[15]
    prob['Pitch_over_duration'] = x[16]
    prob['Delta_vertical_phase'] = x[17]
    prob['Delta_theta_pitch_over'] = x[18]
    prob['command_stage_1_exo'] = x[19:21]
    
    # Fixed parameters
    prob['Diameter_stage_1'] = 5.0
    prob['Diameter_stage_2'] = 5.0
    prob['Mass_flow_rate_stage_1'] = 250.
    prob['Mass_flow_rate_stage_2'] = 250.
    prob['Thrust_stage_1'] = 1000.
    prob['Thrust_stage_2'] = 800.
    prob['payload_mass'] = 5000.0
    
    # FPI for Pdyn
    error = 100.
    prob['Pdyn_max_dim'] = 36.24
    k = 0
    
    while error > 1. and k < 15:
        try:
            prob.run_model()
            error = abs(prob['Pdyn_max_dim'] - prob['max_pdyn_load_ascent_stage_1']/1e3)
            prob['Pdyn_max_dim'] = prob['max_pdyn_load_ascent_stage_1']/1e3
            k += 1
        except:
            return False, {}
    
    if error > 1.:
        return False, {}
    
    # Extract results
    nb_pts = int(prob['Nb_pt_ascent'][0])
    results = {
        'ESA_score': prob['ESA_single_score'][0],
        'delta_v': prob['V_ascent'][nb_pts-1] - prob['V_ascent'][0],
        'altitude': prob['alt_ascent'][nb_pts-1],
        'GLOW': prob['GLOW'][0],
        'GWP': prob['GWP_total'][0],
        'converged': True
    }
    
    return True, results

In [ ]:
"""
## Baseline Evaluation
"""

# Setup problem
prob = om.Problem()
prob.model = LauncherVehicleGroup()
prob.setup(check=False)

# Evaluate baseline
converged, baseline_results = run_fpi(x_baseline_norm, lowerbnd, upperbnd, prob)

if converged:
    print(f"Baseline ESA Score: {baseline_results['ESA_score']:.1f}")
    print(f"Baseline ΔV: {baseline_results['delta_v']:.0f} m/s")
    print(f"Baseline Altitude: {baseline_results['altitude']/1000:.0f} km")
    print(f"Baseline GLOW: {baseline_results['GLOW']/1000:.1f} t")

In [ ]:
"""
## Method 1: CMA-ES with Augmented Lagrangian
Stochastic global optimization with constraint handling
"""

class AugmentedLagrangian:
    def __init__(self, n_constraints):
        self.gamma = np.zeros(n_constraints)  # Lagrange multipliers
        self.omega = np.ones(n_constraints) * 100.0  # Penalty parameters
        self.omega[:2] = 1000.0  # Stronger for hard constraints
        
    def compute(self, f, g):
        """Compute augmented objective"""
        L = f
        for i in range(len(g)):
            if self.gamma[i] + self.omega[i] * g[i] >= 0:
                L += self.gamma[i] * g[i] + 0.5 * self.omega[i] * g[i]**2
        return L
    
    def update(self, g):
        """Update multipliers"""
        for i in range(len(g)):
            self.gamma[i] = max(0, self.gamma[i] + self.omega[i] * g[i] / 5)

# Objective function for CMA-ES
al_handler = AugmentedLagrangian(n_constraints=4)
eval_count = 0

def objective_cmaes(x):
    global eval_count
    eval_count += 1
    
    converged, results = run_fpi(x, lowerbnd, upperbnd, prob)
    
    if not converged:
        return 1e10
    
    # Constraints (g <= 0)
    g = [
        MIN_DELTA_V - results['delta_v'],      # Hard
        MIN_ALTITUDE - results['altitude'],     # Hard
        results['GLOW'] - MAX_GLOW,            # Soft
        (x[1]/x[0]) - 0.3                      # Soft: prop ratio
    ]
    
    # Progress
    if eval_count % 30 == 0:
        print(f"  Eval {eval_count}: ESA={results['ESA_score']:.1f}, ΔV={results['delta_v']:.0f}")
    
    # Heavy penalty for hard constraint violations
    if g[0] > 0 or g[1] > 0:
        return results['ESA_score'] + 1000 * (max(0, g[0])**2 + max(0, g[1])**2)
    
    return al_handler.compute(results['ESA_score'], g)

# Run CMA-ES
print("Starting CMA-ES optimization...")
t0 = time.time()

res_cmaes = cma.fmin(
    objective_cmaes,
    x_baseline_norm,
    0.2,  # sigma
    {
        'bounds': [np.zeros(len(lowerbnd)), np.ones(len(upperbnd))],
        'popsize': 20,
        'maxiter': 30,
        'verb_disp': 0,
        'seed': 42
    }
)

x_opt_cmaes = res_cmaes[0]
converged, results_cmaes = run_fpi(x_opt_cmaes, lowerbnd, upperbnd, prob)

print(f"\nCMA-ES Complete in {time.time()-t0:.1f}s")
print(f"ESA Score: {results_cmaes['ESA_score']:.1f}")
print(f"ΔV: {results_cmaes['delta_v']:.0f} m/s {'✓' if results_cmaes['delta_v'] >= MIN_DELTA_V else '✗'}")
print(f"Altitude: {results_cmaes['altitude']/1000:.0f} km {'✓' if results_cmaes['altitude'] >= MIN_ALTITUDE else '✗'}")

In [ ]:
"""
## Method 2: SLSQP (Sequential Least Squares Programming)
Gradient-based local optimization for validation
"""

eval_count_slsqp = 0

def objective_slsqp(x):
    global eval_count_slsqp
    eval_count_slsqp += 1
    
    converged, results = run_fpi(x, lowerbnd, upperbnd, prob)
    
    if eval_count_slsqp % 20 == 0:
        print(f"  Eval {eval_count_slsqp}: ESA={results['ESA_score'] if converged else 'fail':.1f}")
    
    return results['ESA_score'] if converged else 1e10

# Constraint functions
def constraint_deltav(x):
    _, results = run_fpi(x, lowerbnd, upperbnd, prob)
    return results['delta_v'] - MIN_DELTA_V if results else -1e10

def constraint_altitude(x):
    _, results = run_fpi(x, lowerbnd, upperbnd, prob)
    return results['altitude'] - MIN_ALTITUDE if results else -1e10

constraints = [
    {'type': 'ineq', 'fun': constraint_deltav},
    {'type': 'ineq', 'fun': constraint_altitude}
]

# Run SLSQP
print("\nStarting SLSQP optimization...")
t0 = time.time()

res_slsqp = minimize(
    objective_slsqp,
    x_baseline_norm,
    method='SLSQP',
    bounds=[(0, 1)] * len(lowerbnd),
    constraints=constraints,
    options={'ftol': 1e-8, 'maxiter': 100, 'disp': False}
)

x_opt_slsqp = res_slsqp.x
converged, results_slsqp = run_fpi(x_opt_slsqp, lowerbnd, upperbnd, prob)

print(f"\nSLSQP Complete in {time.time()-t0:.1f}s")
print(f"ESA Score: {results_slsqp['ESA_score']:.1f}")
print(f"ΔV: {results_slsqp['delta_v']:.0f} m/s {'✓' if results_slsqp['delta_v'] >= MIN_DELTA_V else '✗'}")
print(f"Altitude: {results_slsqp['altitude']/1000:.0f} km {'✓' if results_slsqp['altitude'] >= MIN_ALTITUDE else '✗'}")

In [ ]:
"""
## Results Comparison
"""

# Create comparison DataFrame
comparison = pd.DataFrame({
    'Method': ['Baseline', 'CMA-ES-AL', 'SLSQP'],
    'ESA Score': [
        baseline_results['ESA_score'],
        results_cmaes['ESA_score'],
        results_slsqp['ESA_score']
    ],
    'ΔV (m/s)': [
        baseline_results['delta_v'],
        results_cmaes['delta_v'],
        results_slsqp['delta_v']
    ],
    'Altitude (km)': [
        baseline_results['altitude']/1000,
        results_cmaes['altitude']/1000,
        results_slsqp['altitude']/1000
    ],
    'GLOW (t)': [
        baseline_results['GLOW']/1000,
        results_cmaes['GLOW']/1000,
        results_slsqp['GLOW']/1000
    ],
    'Feasible': [
        baseline_results['delta_v'] >= MIN_DELTA_V and baseline_results['altitude'] >= MIN_ALTITUDE,
        results_cmaes['delta_v'] >= MIN_DELTA_V and results_cmaes['altitude'] >= MIN_ALTITUDE,
        results_slsqp['delta_v'] >= MIN_DELTA_V and results_slsqp['altitude'] >= MIN_ALTITUDE
    ]
})

print(comparison.round(1))

# Find best feasible
feasible_methods = comparison[comparison['Feasible']]
if not feasible_methods.empty:
    best_idx = feasible_methods['ESA Score'].idxmin()
    print(f"\nBest feasible solution: {feasible_methods.loc[best_idx, 'Method']}")
    print(f"ESA Score: {feasible_methods.loc[best_idx, 'ESA Score']:.1f}")

In [ ]:
"""
## Optimal Design Analysis
"""

# Select best solution
if results_cmaes['delta_v'] >= MIN_DELTA_V and results_cmaes['altitude'] >= MIN_ALTITUDE:
    x_best = x_opt_cmaes
    print("Using CMA-ES solution")
elif results_slsqp['delta_v'] >= MIN_DELTA_V and results_slsqp['altitude'] >= MIN_ALTITUDE:
    x_best = x_opt_slsqp
    print("Using SLSQP solution")
else:
    x_best = x_baseline_norm
    print("No feasible solution found, using baseline")

# Denormalize
x_best_actual = lowerbnd + (upperbnd - lowerbnd) * x_best

print("\n=== Optimal Design Variables ===")
print(f"Propellant masses: {x_best_actual[0]:.1f}t / {x_best_actual[1]:.1f}t")
print(f"O/F ratios: {x_best_actual[2]:.2f} / {x_best_actual[3]:.2f}")
print(f"Prop ratio: {x_best_actual[1]/x_best_actual[0]:.3f}")

# Material breakdown
print(f"\nStage 1 materials:")
print(f"  CFRP: {x_best_actual[4]:.2f}")
print(f"  Aluminum: {x_best_actual[5]:.2f}")
print(f"  Steel: {x_best_actual[6]:.2f}")

print(f"\nStage 2 materials:")
print(f"  CFRP: {x_best_actual[7]:.2f}")
print(f"  Aluminum: {x_best_actual[8]:.2f}")
print(f"  Steel: {x_best_actual[9]:.2f}")

In [ ]:
"""
## Trajectory Visualization
"""

# Re-run with best solution for plotting
converged, _ = run_fpi(x_best, lowerbnd, upperbnd, prob)

if converged:
    # Generate plots
    try:
        result_vizualization.plots_output(prob)
        print("Trajectory plots generated")
    except:
        # Basic plots if visualization module fails
        nb_pts = int(prob['Nb_pt_ascent'][0])
        time = prob['T_ascent'][:nb_pts]
        altitude = prob['alt_ascent'][:nb_pts] / 1000
        velocity = prob['V_ascent'][:nb_pts]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        ax1.plot(time, altitude)
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel('Altitude (km)')
        ax1.grid(True)
        ax1.set_title('Altitude Profile')
        
        ax2.plot(time, velocity)
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('Velocity (m/s)')
        ax2.grid(True)
        ax2.set_title('Velocity Profile')
        ax2.axhline(y=MIN_DELTA_V, color='r', linestyle='--', label='Required ΔV')
        ax2.legend()
        
        plt.tight_layout()
        plt.show()

In [ ]:
"""
## Environmental Impact Breakdown
"""

if converged:
    print("=== ESA Category Contributions ===")
    categories = ['GWP', 'ODEPL', 'PMAT', 'LUP', 'WDEPL']
    
    for cat in categories:
        try:
            raw = prob[f'ESA_{cat}'][0]
            normalized = prob[f'ESA_{cat}_normalized'][0]
            print(f"{cat:8} raw={raw:.2e}, normalized={normalized:.2e}")
        except:
            pass
    
    print(f"\nTotal ESA Single Score: {prob['ESA_single_score'][0]:.1f}")
    print(f"Total GWP: {prob['GWP_total'][0]/1e3:.1f} tCO2eq")

In [ ]:
"""
## Save Results
"""

# Save to CSV
comparison.to_csv('optimization_results.csv', index=False)
print("Results saved to optimization_results.csv")

# Save optimal design
optimal_design = pd.DataFrame({
    'Variable': ['Prop_mass_1', 'Prop_mass_2', 'OF_1', 'OF_2'],
    'Value': x_best_actual[:4],
    'Unit': ['t', 't', '-', '-']
})
optimal_design.to_csv('optimal_design.csv', index=False)
print("Optimal design saved to optimal_design.csv")